In [52]:
#libraries
import pandas as pd
import numpy as np
from prettytable import PrettyTable
import seaborn as sns
import matplotlib.pyplot as plt
import re
import regex
import string
from sklearn.feature_extraction.text import CountVectorizer

from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
#installing tweet-preprocessor
from tqdm.auto import tqdm
!pip install tweet-preprocessor --user
# import preprocessor as p
from pathlib import Path

#emoji library
# !pip install -e emoji
# import emoji

# model save
import joblib

#split
from sklearn.model_selection import train_test_split

! pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /home/hemant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hemant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hemant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [53]:
class featurise(object):
    def __init__(self, df):
        super(featurise, self).__init__()
        self.x = df['tweets']
        self.y = df['target']
        self.intensity = df['target_intensity']
        ##########################################
        self.final_df = pd.DataFrame()
        self.final_df['new_tweets'] = self.x
        self.final_df['target'] = self.y
        self.final_df['target_intensity'] = self.intensity
        ##########################################
        self.stop_words = stopwords.words('english')
        self.stop_words.remove('not')
        ##########################################
        self.nrc = pd.read_csv("data/3b/lexicons/7. NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        self.dic = {self.nrc[0][i]:self.nrc[1][i] for i in range(len(self.nrc))}
        self.mpqa_df = pd.read_csv("data/3b/lexicons/mpqa.txt",header=None, sep="\t")
        self.dic_mpqa = self.mpqa_df.set_index(0).to_dict()
        self.bingliu_df = pd.read_csv("data/3b/lexicons/1. BingLiu.csv", header=None, sep="\t")
        self.dic_bing = self.bingliu_df.set_index(0).to_dict()
        self.sent140_df = pd.read_csv("data/3b/lexicons/3. Sentiment140-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        self.dic_sent = self.sent140_df.set_index(0).to_dict()
        self.nrc_hash_df = pd.read_csv("data/3b/lexicons/7. NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        self.dic_nrchash = self.nrc_hash_df.set_index(0).to_dict()
        self.nrc_emotion_df = pd.read_csv("data/3b/lexicons/7. NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        self.dic_nrc_emotion = self.nrc_emotion_df.set_index(0).to_dict()
        
        self.nrc_word_emotion_df = pd.read_csv("data/3b/lexicons/8. NRC-word-emotion-lexicon.txt", header=None, sep="\t")
        self.dic_word_nrc_emotion = self.nrc_word_emotion_df.set_index(0).to_dict()
        
        self.afinn_word_df = pd.read_csv("data/3b/lexicons/AFINN-111.txt", header=None, sep="\t")
        self.dic_afinn_word = self.afinn_word_df.set_index(0).to_dict()
        
        df = pd.read_csv("data/3b/lexicons/6. NRC-10-expanded.csv", sep="\t")
        df['sum'] = df['anger']+df['anticipation']+df['disgust']+df['fear']+df['joy']+df['negative']+df['positive']+df['sadness']+df['surprise']+df['trust']
        df = df[["word","sum"]]
        self.afinn_emotion_dic = df.set_index('word').to_dict()
        
        self.agg_has_nrcext_df = pd.read_csv("data/3b/lexicons/5. NRC-Hashtag-Emotion-Lexicon-v0.2.txt", header=None, sep="\t")
        
        self.afinn_df = pd.read_csv("data/3b/lexicons/9. AFINN-emoticon-8.txt",header=None, sep="\t")
        self.afinn_dic = self.afinn_df.set_index(0).to_dict()
        
        self.sentiword = "data/3b/lexicons/4. SentiWordNet_3.0.0_20130122.txt"
        
        self.negation_w = set(['never', 'no', 'nothing', 'nowhere', 'noone', 'none', 'not', 'havent', 'hasnt', 'hadnt', 'cant', 
                         'couldnt', 'shouldnt', 'wont', 'wouldnt', 'dont', 'doesnt', 'didnt', 'isnt', 'arent', 'aint'] )
        
        ############################################################################################################
        
    def valder_scores(self, x):
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(x)
        self.final_df['vn'] = sentiment_dict['neg']
        self.final_df['vp'] = sentiment_dict['pos']
        self.final_df['vneu'] = sentiment_dict['neu']
        self.final_df['comp'] = sentiment_dict['compound']

        # return np.array([sentiment_dict['neg'], sentiment_dict['neu'], sentiment_dict['pos'], sentiment_dict['compound']])
        return self

    def polar_word_count_mpqa(self, x):
        pos,neg = 0,0
        for each in  x.split():
            if each.lower() in self.dic_mpqa[1]:
                if self.dic_mpqa[1][each.lower()] == 'negative': neg+=1
                else: pos+=1
        return pos,neg


    def polar_word_count_bingliu(self, x):
        pos,neg = 0,0
        for each in  x.split():
            if each.lower() in self.dic_bing[1]:
                if self.dic_bing[1][each.lower()] == 'negative': neg+=1
                else: pos+=1
        return pos,neg
    

    def agg_polarity_score_sent140(self,x):
        
        pos,neg = 0,0
        for each in  x.lower().split():
            if each in self.dic_sent[2]:
                pos+=self.dic_sent[2][each]
            if each in self.dic_sent[3]:
                neg+=self.dic_sent[3][each]
        return pos, neg

    def nrc_hashtag_score(self,x):
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in self.dic_nrchash[1].keys():
                score+=self.dic_nrchash[1][each]
        return score

    def nrc_emotion_score(self,x):
        score = 0
        
        for each in  word_tokenize(x.lower()):
            if each in self.dic_nrc_emotion[1].keys():
                score+=self.dic_nrc_emotion[1][each]
        return score

    def nrc_word_emotion(self,x):
        # df = pd.read_csv("data/3b/lexicons/8. NRC-word-emotion-lexicon.txt", header=None, sep="\t")
        # dic = df.set_index(0).to_dict()
        #use tis self.dic_word_nrc_emotion[1]
        score = 0
        # do stuff prepare a df like bow and compute the freq of each emotion
        return score

    def affin_word_score(self,x):
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in self.dic_afinn_word[1].keys():
                score+=self.dic_afinn_word[1][each]
        return score

    def agg_emotion_nrc10ext(self,x):
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in self.afinn_emotion_dic['sum'].keys():
                score+=self.afinn_emotion_dic['sum'][each]
        return score

#     def sentiwordnet(self,x):

#         return None

    def agg_hash_nrc10ext(self,x):
        df = self.agg_has_nrcext_df
        score = 0
        for each in  word_tokenize(x.lower()):
            if each in set(df[1]):
                score+=sum(list(df[df[1]==str(each)][2]))
        return score
    
    def emoticon_afinn(self,x):
        dic,df = self.afinn_dic,self.afinn_df
        score = 0
        for each in  word_tokenize(x):
            if each in df[1]:
                score+=dic[1][each]
        return score

    def negation(self, x):
        splits = ",.:;!?"
        final,flag,count = [],0,0
#         x = f"I don't think I will enjoy it: it might be too spicy.".lower()
        for word in x.split():
            if flag ==1:
                for symb in splits:
                    if symb in word:
                        flag = 0
                        word_split = word.split(symb)
                        if len(word_split[0])>0:
                            count+=1
                            if len(word_split[-1])>1: pass
                        else: pass
                if flag==1: count+=1
            elif flag==0: pass
            clean = re.sub('[^A-Za-z0-9]+', '', word)
            if clean in self.negation_w:
                flag=1
        return count

    def all_lower(self, x):
        return x.lower()
    
    def process(self):
        self.final_df['new_tweets'] = tqdm(self.final_df['new_tweets'].map(self.all_lower))
        self.x.map(self.valder_scores)
        self.final_df['negation'] = [self.negation(i)  for i in tqdm(self.x)]
        
        dummy = np.array([self.polar_word_count_mpqa(i) for i in tqdm(self.x)])
        self.final_df["pos_mpqa"] = dummy[:,0]
        self.final_df["neg_mpqa"] = dummy[:,1]

        dummy = np.array([self.polar_word_count_bingliu(i) for i in tqdm(self.x)])
        self.final_df['pos_bilingu'] = dummy[:,0]
        self.final_df['neg_bilingu'] = dummy[:,1]
        
        dummy = np.array([self.agg_polarity_score_sent140(i) for i in tqdm(self.x)])
        self.final_df['pos_sent140'] = dummy[:,0]
        self.final_df['neg_sent140'] = dummy[:,1]

        self.final_df['nrc_hash'] = tqdm(self.final_df['new_tweets'].map(self.nrc_hashtag_score))

        self.final_df['nrc_emotion_score'] = tqdm(self.final_df['new_tweets'].map(self.nrc_emotion_score))
#         self.final_df['nrc_word_emotion'] = tqdm(self.final_df['new_tweets'].map(self.nrc_word_emotion))
        self.final_df['affin_word_score'] = tqdm(self.final_df['new_tweets'].map(self.affin_word_score))
        self.final_df['agg_emotion_nrc10ext'] = tqdm(self.final_df['new_tweets'].map(self.agg_emotion_nrc10ext))
#         self.final_df['sentiwordnet'] = tqdm(self.final_df['new_tweets'].map(self.sentiwordnet))
        self.final_df['agg_hash_nrc10ext'] = tqdm(self.final_df['new_tweets'].map(self.agg_hash_nrc10ext))
        self.final_df['emoticon_afinn'] = tqdm(self.final_df['new_tweets'].map(self.emoticon_afinn))

        return self


#unigrams bow
def bow(X_train, X_test, X_cv, y_train, y_test, y_cv):

    count_vect = CountVectorizer() #in scikit-learn
    count_vect.fit(X_train)

    bow_counts_train = count_vect.transform(X_train)
    bow_counts_cv = count_vect.transform(X_cv)
    bow_counts_test = count_vect.transform(X_test)
    print("the shape of out text BOW vectorizer ",bow_counts_train.get_shape())
    print("the number of unique words ", bow_counts_train.get_shape()[1])
    # final_counts_test=final_counts_test.toarray()
    return bow_counts_train, bow_counts_cv, bow_counts_test

#bigrams bow
def bigram(X_train, X_test, X_cv, y_train, y_test, y_cv):
    count_vect = CountVectorizer(ngram_range=(2,2))
    bigram_counts_train = count_vect.fit_transform(X_train)
    bigram_counts_cv = count_vect.transform(X_cv)
    bigram_counts_test = count_vect.transform(X_test)
    print("some feature names ", count_vect.get_feature_names()[:10])
    print("the shape of out  BIGRAM vectorizer ",bigram_counts_train.get_shape())
    print("the number of unique words with bigrams ", bigram_counts_train.get_shape()[1])
    return bigram_counts_train, bigram_counts_cv, bigram_counts_test



In [54]:
columns = ['id', "tweets", "target", "target_intensity"]
joy_train = pd.read_csv("data/3b/lexicons/joy-ratings-0to1.train.txt", sep="\t", header=None, names=columns)
joy_test = pd.read_csv("data/3b/lexicons/joy-ratings-0to1.test.gold.txt", sep="\t", header=None, names=columns)
anger_train = pd.read_csv("data/3b/lexicons/anger-ratings-0to1.train.txt", sep="\t", header=None, names=columns)
anger_test = pd.read_csv("data/3b/lexicons/anger-ratings-0to1.test.gold.txt", sep="\t", header=None, names=columns)

train = pd.concat([anger_train,joy_train]).sample(frac=1).reset_index(drop=True)
test = pd.concat([anger_test,joy_test]).sample(frac=1).reset_index(drop=True)

In [55]:
### FEATURISATION

flag = 0

my_file = Path("data/3b/train_df_3B.csv")
if flag and my_file.is_file():
    print("CSV Found")
    train_df = pd.read_csv("data/3b/train_df_3B.csv")
else:
    print("Generating CSV")
    # featurise train data
    proc = featurise(train).process()
    train_df = proc.final_df
    train_df.to_csv('data/3b/train_df_3B.csv')

my_file = Path("data/3b/test_df.csv_3B")
if flag and my_file.is_file():
    print("CSV Found")
    test_df = pd.read_csv("data/3b/test_df_3B.csv")
else:
    print("Generating CSV")
    # featurise test data
    proc = featurise(test).process()
    test_df = proc.final_df
    test_df.to_csv('data/3b/test_df_3B.csv')

Generating CSV



Generating CSV


In [56]:
y_train = train_df['target']
X_train = train_df.drop(['target'],axis=1)
y_test = test_df['target']
X_test = test_df.drop(['target'],axis=1)

X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

bow_counts_train, bow_counts_cv, bow_counts_test = bow(X_train['new_tweets'], X_test['new_tweets'], X_cv['new_tweets'], y_train, y_test, y_cv)
bigram_counts_train, bigram_counts_cv, bigram_counts_test = bigram(X_train['new_tweets'], X_test['new_tweets'], X_train['new_tweets'], y_train, y_test, y_cv)

the shape of out text BOW vectorizer  (1125, 4449)
the number of unique words  4449
some feature names  ['00 dollars', '10 11', '10 because', '10 for', '10 minutes', '100 real', '100 would', '1000 rabid', '1000 subscribers', '100s of']
the shape of out  BIGRAM vectorizer  (1125, 11479)
the number of unique words with bigrams  11479


In [57]:
from scipy import sparse
def save_npz(f,name):
    sparse.save_npz("data/3b/npz/"+str(name)+".npz", f)
    return True

save_npz(bow_counts_cv, "bow_counts_cv")
save_npz(bow_counts_test, "bow_counts_test")
save_npz(bow_counts_train, "bow_counts_train")
save_npz(bigram_counts_train, "bigram_counts_train")
save_npz(bigram_counts_test, "bigram_counts_test")
save_npz(bigram_counts_cv, "bigram_counts_cv")

# train_df.to_csv('data/3a/gram_traincsv.csv')
# val_df.to_csv('data/3a/gram_valcsv.csv')
# test_df.to_csv('data/3a/gram_testcsv.csv')

True

In [69]:
tra = np.concatenate((bigram_counts_train.toarray(),bigram_counts_cv.toarray()))
tes =bigram_counts_test.toarray()

# uni_t  = pd.DataFrame(tra)
# uni_v = pd.DataFrame(tes)

bi_t  = pd.DataFrame(tra)
bi_v = pd.DataFrame(tes)

In [76]:
bi_t.to_csv('data/3a/bi_t.csv')

In [ ]:
bow_counts_test.shape

In [22]:
bow_counts_cv.toarray().shape


(33, 579)

SENTI Word

In [58]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

fp = "data/3b/lexicons/4. SentiWordNet_3.0.0_20130122.txt"
f = open(fp)    

def split_line(line):
    cols = line.split("\t")
    return cols
def get_words(cols):
    words_ids = cols[4].split(" ")
    words = [w.split("#")[0] for w in words_ids]
    return words
def get_positive(cols):
    return cols[2]
def get_negative(cols):
    return cols[3]
def get_objective(cols):
    return 1 - (float(cols[2]) + float(cols[3]))
def get_gloss(cols):
    return cols[5]
def get_scores(sentiword):
    totalobject, count, totalpositive, totalnegative =0.0,0.0,0.0,0.0,
    for line in f:
        if not line.startswith("#"):
            cols = split_line(line)
            words = get_words(cols)
            for word in sentiword:
                if word in words:
                    if word == "not":
                        totalobject = totalobject + 0
                        totalpositive = totalpositive + 0
                        totalnegative = totalnegative + 16
                        count =count + 1
                    else:
                        totalobject = totalobject + get_objective(cols)
                        totalpositive = totalpositive + float(get_positive(cols))
                        totalnegative = totalnegative + float(get_negative(cols))
                        count =count + 1
    return totalpositive, totalnegative

def call_senti(sentiword):
    filtered_sentence = [w for w in sentiword  if not w in stop_words ]
    return get_scores(filtered_sentence) 

In [59]:
dummy = np.array([call_senti(i) for i in tqdm(train_df['new_tweets'])])
train_df['senti_pos'] = dummy[:,0]
train_df['senti_neg'] = dummy[:,1]

In [60]:
dummy = np.array([call_senti(i) for i in tqdm(test_df['new_tweets'])])
test_df['senti_pos'] = dummy[:,0]
test_df['senti_neg'] = dummy[:,1]



---

Emotion Word Count

In [61]:
df = pd.read_csv("data/3b/lexicons/8. NRC-word-emotion-lexicon.txt", header=None, sep='\t')
dic = df.groupby(1)[0].apply(set)
col = list(dic.keys().values)

In [62]:
def word_emotion(x):
    x = x.lower().split()
    row = [0]*len(col)
    for each in x:
        for c in col:
            if each in dic[c]:
                # print(each, c)
                row[col.index(c)]+=1
            else:
                # print("---------",each,c)
                pass
    return row

In [63]:
lol = [word_emotion(i) for i in train_df['new_tweets']]
train_df = pd.concat([train_df, pd.DataFrame(lol, columns=col)],axis=1)
train_df.to_csv('data/3b/train_df_3B.csv')

lol = [word_emotion(i) for i in test_df['new_tweets']]
test_df = pd.concat([test_df, pd.DataFrame(lol, columns=col)],axis=1)
test_df.to_csv('data/3b/test_df_3B.csv')